In [3]:
def composite_simpsons_rule(f, a, b, n):
    """
    Composite Simpson's Rule for numerical integration.
    
    Parameters:
    f (function): The integrand, a function of one variable.
    a (float): The lower limit of integration.
    b (float): The upper limit of integration.
    n (int): The number of subintervals (must be even).
    
    Returns:
    float: The approximate value of the integral.
    """
    
    if n % 2 != 0:
        raise ValueError("Number of subintervals (n) must be even.")
    
    h = (b - a) / n
    x = [a + i*h for i in range(n + 1)]
    y = [f(xi) for xi in x]
    
    integral = y[0] + y[-1]  # First and last terms
    integral += 4 * sum(y[i] for i in range(1, n, 2))  # Odd-indexed terms
    integral += 2 * sum(y[i] for i in range(2, n-1, 2))  # Even-indexed terms
    
    integral *= h / 3
    
    return integral

def f(x):
    return 2/(x**2+4)

result = composite_simpsons_rule(f, 0, 2, 6)
print(f"Approximate value of the integral: {result}")


Approximate value of the integral: 0.7853979452340107


In [1]:
import numpy as np

def romberg_integration(f, a, b, max_order):
    """
    Romberg Integration for numerical integration.
    
    Parameters:
    f (function): The integrand, a function of one variable.
    a (float): The lower limit of integration.
    b (float): The upper limit of integration.
    max_order (int): The maximum order of the Romberg method.
    
    Returns:
    float: The approximate value of the integral.
    numpy.ndarray: The Romberg table showing the progression of results.
    """
    
    # Initialize Romberg table
    R = np.zeros((max_order, max_order))

    # Step 1
    # Compute R[0, 0] using the trapezoidal rule with one segment
    h = b - a
    R[0, 0] = 0.5 * h * (f(a) + f(b))

    # Step 3
    # Fill the Romberg table
    for k in range(1, max_order):
        # Compute trapezoidal rule with 2^k segments
        h /= 2
        # Step 4
        sum_f = sum(f(a + (2*i - 1) * h) for i in range(1, 2**(k-1) + 1))
        R[k, 0] = 0.5 * R[k-1, 0] + h * sum_f

        # Step 5
        # Compute the rest of the Romberg table using Richardson extrapolation
        for j in range(1, k + 1):
            R[k, j] = R[k, j-1] + (R[k, j-1] - R[k-1, j-1]) / (4**j - 1)
    
    return R[max_order-1, max_order-1], R

# Example usage
def f(x):
    return x**2

result, romberg_table = romberg_integration(f, 0, 1, 4)
print(f"Approximate value of the integral: {result}")
print("Romberg Table:")
print(romberg_table)


Approximate value of the integral: 0.3333333333333333
Romberg Table:
[[0.5        0.         0.         0.        ]
 [0.375      0.33333333 0.         0.        ]
 [0.34375    0.33333333 0.33333333 0.        ]
 [0.3359375  0.33333333 0.33333333 0.33333333]]


In [9]:
def adaptive_simpsons_rule(f, a, b, tol, max_depth=10, depth=0):
    """
    Adaptive Quadrature using Simpson's Rule.
    
    Parameters:
    f (function): The integrand, a function of one variable.
    a (float): The lower limit of integration.
    b (float): The upper limit of integration.
    tol (float): The desired tolerance.
    max_depth (int): Maximum recursion depth.
    depth (int): Current recursion depth (used internally).
    
    Returns:
    float: The approximate value of the integral.
    """
    
    def simpsons(f, a, b):
        """Simpson's Rule on the interval [a, b]."""
        c = (a + b) / 2.0
        return (b - a) / 6.0 * (f(a) + 4.0 * f(c) + f(b))

    # Step 1
    # Apply Simpson's rule on the whole interval [a, b]
    c = (a + b) / 2.0
    S = simpsons(f, a, b)

    # Step 3
    # Apply Simpson's rule on the two subintervals [a, c] and [c, b]
    S_left = simpsons(f, a, c)
    S_right = simpsons(f, c, b)
    
    # Error estimate (difference between whole interval and sum of subintervals)
    error = abs(S_left + S_right - S)

    # Step 5
    # If error is within tolerance or max depth is reached, return the result
    if error < 15 * tol or depth >= max_depth:
        return S_left + S_right + (S_left + S_right - S) / 15.0
    else:
        # Recursively apply to the subintervals
        return (adaptive_simpsons_rule(f, a, c, tol / 2.0, max_depth, depth + 1) +
                adaptive_simpsons_rule(f, c, b, tol / 2.0, max_depth, depth + 1))

# Example usage
def f(x):
    return x**3

result = adaptive_simpsons_rule(f, 0, 1, tol=1e-6)
print(f"Approximate value of the integral: {result}")


Approximate value of the integral: 0.14285716627879688


In [2]:
import numpy as np

def simpsons_double_integral_non_rect(f, ax, bx, g1, g2, nx, ny):
    """
    Simpson's Rule for double integrals over a non-rectangular region.
    
    Parameters:
    f (function): The integrand, a function of two variables (f(x, y)).
    ax (float): The lower limit of the outer integral (x-axis).
    bx (float): The upper limit of the outer integral (x-axis).
    g1 (function): The lower limit of the inner integral as a function of x.
    g2 (function): The upper limit of the inner integral as a function of x.
    nx (int): Number of subintervals for the outer integral (must be even).
    ny (int): Number of subintervals for the inner integral (must be even).
    
    Returns:
    float: The approximate value of the double integral.
    """
    
    if nx % 2 != 0 or ny % 2 != 0:
        raise ValueError("Both nx and ny must be even for Simpson's Rule.")
    
    hx = (bx - ax) / nx  # Step size in the x direction
    
    # Create grid points for x
    x = np.linspace(ax, bx, nx + 1)
    
    integral = 0.0
    for i in range(nx + 1):
        xi = x[i]
        y_lower = g1(xi)  # Lower bound for the inner integral
        y_upper = g2(xi)  # Upper bound for the inner integral
        hy = (y_upper - y_lower) / ny  # Step size in the y direction
        y = np.linspace(y_lower, y_upper, ny + 1)  # Grid points for y
        
        # Apply Simpson's rule in the y direction
        integral_y = 0.0
        for j in range(ny + 1):
            if j == 0 or j == ny:
                coef_y = 1
            elif j % 2 == 0:
                coef_y = 2
            else:
                coef_y = 4
            integral_y += coef_y * f(xi, y[j])
        
        integral_y *= hy / 3.0  # Apply Simpson's Rule scaling factor for y
        
        # Apply Simpson's rule for x direction coefficients
        if i == 0 or i == nx:
            coef_x = 1
        elif i % 2 == 0:
            coef_x = 2
        else:
            coef_x = 4
        
        integral += coef_x * integral_y
    
    integral *= hx / 3.0  # Apply Simpson's Rule scaling factor for x
    
    return integral

# Example usage

# Define the bounds for the non-rectangular region
def g1(x):
    return 0  # Lower bound for y (could be any function of x)

def g2(x):
    return 1 - x  # Upper bound for y (non-rectangular region)

# Define the integrand function
def f(x, y):
    return x * y  # Example function f(x, y) = x * y

# Double integral of f(x, y) over the non-rectangular region
# Here we integrate over 0 <= x <= 1 and 0 <= y <= 1 - x
result = simpsons_double_integral_non_rect(f, 0, 1, g1, g2, nx=10, ny=10)
print(f"Approximate value of the double integral: {result}")


Approximate value of the double integral: 0.04166666666666666


In [4]:
import numpy as np
import scipy.special

def gaussian_double_integral_non_rect(f, ax, bx, g1, g2, nx, ny):
    """
    Gaussian quadrature for double integrals over a non-rectangular region.
    
    Parameters:
    f (function): The integrand, a function of two variables (f(x, y)).
    ax (float): The lower limit of the outer integral (x-axis).
    bx (float): The upper limit of the outer integral (x-axis).
    g1 (function): The lower limit of the inner integral as a function of x.
    g2 (function): The upper limit of the inner integral as a function of x.
    nx (int): Number of quadrature points for the outer integral.
    ny (int): Number of quadrature points for the inner integral.
    
    Returns:
    float: The approximate value of the double integral.
    """
    
    # Get the Gaussian quadrature nodes and weights for x and y
    x_nodes, x_weights = np.polynomial.legendre.leggauss(nx)  # For outer integral (x)
    
    # Map the x_nodes from the interval [-1, 1] to [ax, bx]
    x_mapped = 0.5 * (bx - ax) * x_nodes + 0.5 * (bx + ax)
    x_weights_mapped = 0.5 * (bx - ax) * x_weights
    
    total_integral = 0.0
    
    for i, xi in enumerate(x_mapped):
        y_lower = g1(xi)  # Lower bound for the inner integral (y)
        y_upper = g2(xi)  # Upper bound for the inner integral (y)
        
        # Get the Gaussian quadrature nodes and weights for y
        y_nodes, y_weights = np.polynomial.legendre.leggauss(ny)
        
        # Map the y_nodes from [-1, 1] to [g1(x), g2(x)]
        y_mapped = 0.5 * (y_upper - y_lower) * y_nodes + 0.5 * (y_upper + y_lower)
        y_weights_mapped = 0.5 * (y_upper - y_lower) * y_weights
        
        # Perform the Gaussian quadrature for the inner integral (y)
        inner_integral = 0.0
        for j, yj in enumerate(y_mapped):
            inner_integral += y_weights_mapped[j] * f(xi, yj)
        
        # Multiply by the weight for the outer integral (x)
        total_integral += x_weights_mapped[i] * inner_integral
    
    return total_integral

# Example usage

# Define the bounds for the non-rectangular region
def g1(x):
    return 0  # Lower bound for y

def g2(x):
    return 1 - x  # Upper bound for y (non-rectangular region)

# Define the integrand function
def f(x, y):
    return x * y  # Example function f(x, y) = x * y

# Double integral of f(x, y) over the non-rectangular region
# Here we integrate over 0 <= x <= 1 and 0 <= y <= 1 - x
result = gaussian_double_integral_non_rect(f, 0, 1, g1, g2, nx=10, ny=10)
print(f"Approximate value of the double integral: {result}")


Approximate value of the double integral: 0.04166666666666668
